In [1]:
import CNN2Head_input
import tensorflow as tf
import numpy as np

SAVE_FOLDER = '/home/ubuntu/coding/cnn/multi-task-learning/save/current'

_, smile_test_data =  CNN2Head_input.getSmileImage()
_, gender_test_data =  CNN2Head_input.getGenderImage()
_, age_test_data =  CNN2Head_input.getAgeImage()

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


Load smile image...................
Done !
Number of smile train data:  3000
---------------------------------------------------------------
Load gender image...................
Done !
Number of gender train data:  150000
---------------------------------------------------------------
Load age image...................
Done !
Number of age train data:  150000
---------------------------------------------------------------


In [3]:
def eval_smile_gender_age_test(nbof_crop):
    nbof_smile = len(smile_test_data)
    nbof_gender = len(gender_test_data)
    nbof_age = len(age_test_data)

    nbof_true_smile = 0
    nbof_true_gender = 0
    nbof_true_age = 0

    sess = tf.InteractiveSession()
    saver = tf.train.import_meta_graph(SAVE_FOLDER + '/model.ckpt.meta')
    saver.restore(sess, SAVE_FOLDER + "/model.ckpt")
    
    x_smile = tf.get_collection('x_smile')[0]
    x_gender = tf.get_collection('x_gender')[0]
    x_age = tf.get_collection('x_age')[0]
    
    keep_prob_smile_fc1 = tf.get_collection('keep_prob_smile_fc1')[0]
    keep_prob_gender_fc1 = tf.get_collection('keep_prob_gender_fc1')[0]
    keep_prob_age_fc1 = tf.get_collection('keep_prob_age_fc1')[0]
    
    keep_prob_smile_fc2 = tf.get_collection('keep_prob_smile_fc2')[0]
    keep_prob_gender_fc2 = tf.get_collection('keep_prob_emotion_fc2')[0]
    keep_prob_age_fc2 = tf.get_collection('keep_prob_age_fc2')[0]
    
    y_smile_conv = tf.get_collection('y_smile_conv')[0]
    y_gender_conv = tf.get_collection('y_gender_conv')[0]
    y_age_conv = tf.get_collection('y_age_conv')[0]
    
    is_training = tf.get_collection('is_training')[0]

    for i in range(nbof_smile):        
        smile = np.zeros([1,48,48,1])
        smile[0] = smile_test_data[i % 1000][0]
        smile_label = smile_test_data[i % 1000][1]
        
        gender = np.zeros([1,48,48,1])
        gender[0] = gender_test_data[i % 1000][0]
        gender_label = gender_test_data[i % 1000][1]
        
        age = np.zeros([1,48,48,1])
        age[0] = age_test_data[i % 1000][0]
        age_label = age_test_data[i % 1000][1]
        
        y_smile_pred = np.zeros([2])
        y_gender_pred = np.zeros([2])
        y_age_pred = np.zeros([4])

        for _ in range(nbof_crop):
            x_smile_ = CNN2Head_input.random_crop(smile, (48, 48), 10)
            x_gender_ = CNN2Head_input.random_crop(gender,(48, 48), 10)
            x_age_ = CNN2Head_input.random_crop(age,(48, 48), 10)
            
            y1 = y_smile_conv.eval(feed_dict={x_smile: x_smile_,
                                              x_gender: x_gender_,
                                              x_age: x_age_,
                                              keep_prob_smile_fc1: 1,
                                              keep_prob_smile_fc2: 1,
                                              keep_prob_gender_fc1: 1,
                                              keep_prob_gender_fc2: 1,
                                              keep_prob_age_fc1: 1,
                                              keep_prob_age_fc2: 1,
                                              is_training: False})
            y2 = y_gender_conv.eval(feed_dict={x_smile: x_smile_,
                                               x_gender: x_gender_,
                                               x_age: x_age_,
                                               keep_prob_smile_fc1: 1,
                                               keep_prob_smile_fc2: 1,
                                               keep_prob_gender_fc1: 1,
                                               keep_prob_gender_fc2: 1,
                                               keep_prob_age_fc1: 1,
                                               keep_prob_age_fc2: 1,
                                               is_training: False})
            y3 = y_age_conv.eval(feed_dict={x_smile: x_smile_,
                                            x_gender: x_gender_,
                                            x_age: x_age_,
                                            keep_prob_smile_fc1: 1,
                                            keep_prob_smile_fc2: 1,
                                            keep_prob_gender_fc1: 1,
                                            keep_prob_gender_fc2: 1,
                                            keep_prob_age_fc1: 1,
                                            keep_prob_age_fc2: 1,
                                            is_training: False})
              
            y_smile_pred += y1[0]
            y_gender_pred += y2[0]
            y_age_pred += y3[0]

        predict_smile = np.argmax(y_smile_pred)
        predict_gender = np.argmax(y_gender_pred)
        predict_age = np.argmax(y_age_pred)

        if (predict_smile == smile_label) & (i < 1000):
            nbof_true_smile += 1
        if (predict_gender == gender_label):
            nbof_true_gender += 1
        if (predict_age == age_label):
            nbof_true_age += 1
        
    return nbof_true_smile * 100.0 / nbof_smile, nbof_true_gender * 100.0 / nbof_gender, nbof_true_age * 100.0 / nbof_age


def evaluate(nbof_crop):
    print('Testing phase...............................')
    smile_acc, gender_acc, age_acc = eval_smile_gender_age_test(nbof_crop)
    print('Smile test accuracy: ',str(smile_acc))
    print('Gender test accuracy: ', str(gender_acc))
    print('Age test accuracy: ', str(age_acc))

evaluate(10)

Testing phase...............................
INFO:tensorflow:Restoring parameters from /home/ubuntu/coding/cnn/multi-task-learning/save/current/model.ckpt


IndexError: list index out of range

In [4]:
nbof_smile = len(smile_test_data)
nbof_gender = len(gender_test_data)
nbof_age = len(age_test_data)

nbof_true_smile = 0
nbof_true_gender = 0
nbof_true_age = 0

sess = tf.InteractiveSession()
saver = tf.train.import_meta_graph(SAVE_FOLDER + '/model.ckpt.meta')
saver.restore(sess, SAVE_FOLDER + "/model.ckpt")


INFO:tensorflow:Restoring parameters from /home/ubuntu/coding/cnn/multi-task-learning/save/current/model.ckpt


In [8]:
tf.get_collection('y_smile')[0]

<tf.Tensor 'Slice:0' shape=(128, 2) dtype=float32>